In [1]:
import numpy as np 
import pandas as pd

import os
print(os.listdir("../input"))



/home/raghuvardhan/anaconda3/envs/DS_common/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/raghuvardhan/anaconda3/envs/DS_common/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


['test.csv', 'train.csv']


In [2]:
train_df = pd.read_csv('../input/train.csv')
train_df['target'].head()

0    38000000.0
1      600000.0
2    10000000.0
3     2000000.0
4    14400000.0
Name: target, dtype: float64

In [3]:
train_df.columns

Index(['ID', 'target', '48df886f9', '0deb4b6a8', '34b15f335', 'a8cb14b00',
       '2f0771a37', '30347e683', 'd08d1fbe3', '6ee66e115',
       ...
       '3ecc09859', '9281abeea', '8675bec0b', '3a13ed79a', 'f677d4d13',
       '71b203550', '137efaa80', 'fb36b89d9', '7e293fbaf', '9fc776466'],
      dtype='object', length=4993)

In [4]:
train_df.shape

(4459, 4993)

In [5]:
#remove rows with missing values
train_df = train_df.dropna(axis=0)
train_df.shape

(4459, 4993)

In [6]:
train_df.shape

(4459, 4993)

In [7]:
x = train_df.drop(['target','ID'],axis=1)
y = train_df['target']

In [8]:
x['_mean'] = np.log1p(x.replace(0, np.nan).mean(axis=1))
x['log_median'] = np.log1p(x.replace(0, np.nan).median(axis=1))
x['_sum'] = np.log1p(x.sum(axis=1))
x['_std'] = x.std(axis=1)
x['_kur'] = x.kurtosis(axis=1)

In [9]:
x.columns

Index(['48df886f9', '0deb4b6a8', '34b15f335', 'a8cb14b00', '2f0771a37',
       '30347e683', 'd08d1fbe3', '6ee66e115', '20aa07010', 'dc5a8f1d8',
       ...
       '71b203550', '137efaa80', 'fb36b89d9', '7e293fbaf', '9fc776466',
       '_mean', 'log_median', '_sum', '_std', '_kur'],
      dtype='object', length=4996)

In [10]:
from sklearn.preprocessing import RobustScaler
rscaler = RobustScaler()
feature_normal = rscaler.fit_transform(x)

In [11]:
print(feature_normal.shape)
print(type(feature_normal))

(4459, 4996)
<class 'numpy.ndarray'>


In [12]:
feature_normal = pd.DataFrame(data=feature_normal,columns=x.columns)

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(feature_normal, y, test_size=0.3, random_state=42)

In [14]:
print(x_train.shape)
print(y_train.shape)

(3121, 4996)
(3121,)


In [15]:
print(x_test.shape)
print(y_test.shape)

(1338, 4996)
(1338,)


In [16]:
from sklearn.ensemble import GradientBoostingRegressor

regr = GradientBoostingRegressor(loss='lad',learning_rate=0.01,max_depth=16,n_estimators=400)
regr.fit(x_train, y_train)

/home/raghuvardhan/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='lad', max_depth=16,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=400,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [17]:
from sklearn.metrics import r2_score
pred = regr.predict(x_test)
corr =  np.corrcoef(x=pred,y=y_test)
score = r2_score(y_test,pred)
print("corr={},score={}".format(corr[0,1],score))

corr=0.6002213446989808,score=0.35071723322251824


In [18]:
print(pred.shape)
print(type(pred))

(1338,)
<class 'numpy.ndarray'>


In [19]:
sum(pred <0)

0

In [20]:
regr.fit(x, y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='lad', max_depth=16,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=400,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [21]:
#test_sample = rscaler.fit_transform(test_df)

In [22]:
test_df_full = pd.read_csv('../input/test.csv')
test_df_full.columns

Index(['ID', '48df886f9', '0deb4b6a8', '34b15f335', 'a8cb14b00', '2f0771a37',
       '30347e683', 'd08d1fbe3', '6ee66e115', '20aa07010',
       ...
       '3ecc09859', '9281abeea', '8675bec0b', '3a13ed79a', 'f677d4d13',
       '71b203550', '137efaa80', 'fb36b89d9', '7e293fbaf', '9fc776466'],
      dtype='object', length=4992)

In [23]:
test_df = test_df_full.drop(['ID'],axis=1)
test_df.columns

Index(['48df886f9', '0deb4b6a8', '34b15f335', 'a8cb14b00', '2f0771a37',
       '30347e683', 'd08d1fbe3', '6ee66e115', '20aa07010', 'dc5a8f1d8',
       ...
       '3ecc09859', '9281abeea', '8675bec0b', '3a13ed79a', 'f677d4d13',
       '71b203550', '137efaa80', 'fb36b89d9', '7e293fbaf', '9fc776466'],
      dtype='object', length=4991)

In [24]:
test_df['_mean'] = np.log1p(test_df.replace(0, np.nan).mean(axis=1))
test_df['log_median'] = np.log1p(test_df.replace(0, np.nan).median(axis=1))
test_df['_sum'] = np.log1p(test_df.sum(axis=1))
test_df['_std'] = test_df.std(axis=1)
test_df['_kur'] = test_df.kurtosis(axis=1)

In [25]:
result=regr.predict(test_df)
print(result)

[ 4479175.58511439  2206625.29022893  3869659.68915223 ...
  3962158.1218386   1514589.92414449 10697120.77682394]


In [26]:
sum(result<0)

0

In [27]:
result_df = pd.DataFrame()
result_df['ID']= test_df_full['ID']
result_df['target']= result
outfiles='valueprediction.csv'
result_df.to_csv(path_or_buf=outfiles,index=False)